In [601]:
# Chapter 7; 데이터 정제 및 준비

In [602]:
# 이 장에서는 결측치, 중복 데이터, 문자열 처리 그리고 다른 분석적 데이터 변환에 대한 도구들을 다룬다.

In [603]:
# 다음 장에서는 데이터를 합치고 재배열하는 다양한 방법을 알아보겠다.

In [604]:
# 7.1 누락된 데이터 처리하기

In [605]:
# 누락된 데이터를 처리하는 일은 데이터 분석 애플리케이션에서 흔히 발생하는 일이다.

In [606]:
# pandas의 설계 목표 중 하나는 누락 데이터를 가능한 한 쉽게 처리할 수 있도록 하는 것이다. 

In [607]:
# 예를 들어 pandas 객체의 모든 기술 통계는 누락된 데이터를 베제하고 처리한다.

In [608]:
# pandas 객체에서 누락된 값을 표현하는 방식은 완벽하다고 할 수 없다.

In [609]:
# 산술 데이터에 한해 pandas는 누락된 데이터를 실숫값인 NaN으로 취급한다. 

In [610]:
# 이는 누락된 값을 쉽게 찾을 수 있도록 하는 파수병 역할을 한다.

In [611]:
import numpy as np
import pandas as pd
import sqlalchemy as sqla

In [612]:
string_data = pd.Series(["aardvark", "artichoke", np.nan, "avocado"])

In [613]:
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [614]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In [615]:
# pandas에서는 R 프로그래밍 언어에서 결측치를 NA로 취급하는 개념을 차용했다. 

In [616]:
# 여기서 결측값은 존재하지 않는(null) 값의 일종으로, 변수(variable) 등이며 값은 존재하는 것이지만, 그 시점에서 아직 정해져 있지 않은 값을 표시한다.

In [617]:
# 분석 애플리케이션에서 NA 데이터는 데이터가 존재하지 않거나, 존재하더라도 데이터를 수집하는 과정 등에서 검출되지 않았음을 의미한다. 

In [618]:
# 분석을 위해 데이터를 정제하는 과정에서 결측치 자체를 데이터 수집 과정에서의 실수나 결측치로 인한 잠재적인 편향을 찾아내는 수단으로 인식하는 것은 중요하다.

In [619]:
# 파이썬의 내장 None 값 또한 NA 값으로 취급된다.

In [620]:
string_data[0] = None

In [621]:
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

In [622]:
# pandas 프로젝트에서는 결측치를 처리하는 방법을 개선하는 작업이 진행 중이지만 pandas.isnull 같은 사용자 API 함수에서는 성가신 부분을 추상화로 제거했다.

In [623]:
# [표 7-1]에 결측치 처리와 관련된 함수를 정리해두었다. [표 7-1] NA 처리 메서드 페이지 271쪽

In [624]:
# 7.1.1 누락된 데이터 골라내기

In [625]:
# 누락된 데이터를 골라내는 몇 가지 방법이 있는데, pandas.isnull이나 불리언 색인을 사용해 직접 손으로 제거하는 것도 한 가지 방법이다.

In [626]:
# 하지만 dropna를 매우 유용하게 사용할 수 있다.

In [627]:
# Series에 dropna 메서드를 적용하면 Null이 아닌 데이터와 색인값만 들어 있는 Series를 반환한다.

In [628]:
from numpy import nan as NA

In [629]:
data = pd.Series([1, NA, 3.5, NA, 7])

In [630]:
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [631]:
# 위 코드는 다음과 동일하다.

In [632]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

In [633]:
# DataFrame 객체의 경우에는 조금 복잡한데, 모두 NA 값인 로우나 컬럼을 제외시키거나 NA 값을 하나라도 포함하고 있는 로우나 컬럼을 제외시킬 수 있다.

In [634]:
# dropna는 기본적으로 NA 값을 하나라도 포함하고 있는 로우를 제외시킨다.

In [635]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]])

In [636]:
cleaned = data.dropna()

In [637]:
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [638]:
cleaned

,0,1,2
0,1.0,6.5,3.0


In [639]:
# how="all" 옵션을 넘기면 모두 NA 값인 로우만 제외시킨다.

In [640]:
data.dropna(how="all")

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [641]:
# 컬럼을 제외시키는 방법도 동일하게 동작한다. 옵션으로 axis=1을 넘겨주면 된다.

In [642]:
data[4] = NA

In [643]:
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [644]:
data.dropna(axis=1, how="all")

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [645]:
# DataFrame의 로우를 제외시키는 방법은 시계열 데이터에 주로 사용되는 경향이 있다. 

In [646]:
# 몇 개 이상의 값이 들어 있는 로우만 살펴보고 싶다면 thresh 인자에 원하는 값을 넘기면 된다. 

In [647]:
df = pd.DataFrame(np.random.randn(7, 3))

In [648]:
df.iloc[:4, 1] = NA

In [649]:
df.iloc[:2, 2] = NA

In [650]:
df

,0,1,2
0,1.007189,NaN,NaN
1,0.228913,NaN,NaN
2,-2.001637,NaN,1.669025
3,-0.438570,NaN,0.476985
4,3.248944,-1.021228,-0.577087
5,0.124121,0.302614,0.523772
6,0.000940,1.343810,-0.713544


In [651]:
df.dropna()

,0,1,2
4,3.248944,-1.021228,-0.577087
5,0.124121,0.302614,0.523772
6,0.000940,1.343810,-0.713544


In [652]:
df.dropna(thresh=2)

,0,1,2
2,-2.001637,NaN,1.669025
3,-0.438570,NaN,0.476985
4,3.248944,-1.021228,-0.577087
5,0.124121,0.302614,0.523772
6,0.000940,1.343810,-0.713544


In [653]:
# 7.1.2 결측치 채우기

In [654]:
# 누락된 값을 제외시키지 않고(잠재적으로 다른 데이터도 함께 버려질 가능성이 있다) 데이터 상의 "구멍"을 어떻게든 메우고 싶은 경향이 있다. 

In [655]:
# 이 경우 fillna 메서드를 활용하면 되는데, fillna 메서드에 채워 넣고 싶은 값을 넘겨주면 된다.

In [656]:
df.fillna(0)

,0,1,2
0,1.007189,0.000000,0.000000
1,0.228913,0.000000,0.000000
2,-2.001637,0.000000,1.669025
3,-0.438570,0.000000,0.476985
4,3.248944,-1.021228,-0.577087
5,0.124121,0.302614,0.523772
6,0.000940,1.343810,-0.713544


In [657]:
# fillna에 사전값을 넘겨서 각 컬럼마다 다른 값을 채울 수도 있다.

In [658]:
df.fillna({1:0.5, 2: 0})

,0,1,2
0,1.007189,0.500000,0.000000
1,0.228913,0.500000,0.000000
2,-2.001637,0.500000,1.669025
3,-0.438570,0.500000,0.476985
4,3.248944,-1.021228,-0.577087
5,0.124121,0.302614,0.523772
6,0.000940,1.343810,-0.713544


In [659]:
# fillna는 새로운 객체를 반환하지만 다음처럼 기존 객체를 변경할 수도 있다.

In [660]:
_ = df.fillna(0, inplace=True)

In [661]:
df

,0,1,2
0,1.007189,0.000000,0.000000
1,0.228913,0.000000,0.000000
2,-2.001637,0.000000,1.669025
3,-0.438570,0.000000,0.476985
4,3.248944,-1.021228,-0.577087
5,0.124121,0.302614,0.523772
6,0.000940,1.343810,-0.713544


In [662]:
# 재색인에서 사용 가능한 보간 메서드는 fillna 메서드에서도 사용가능하다.

In [663]:
df = pd.DataFrame(np.random.randn(6, 3))

In [664]:
df.iloc[2:, 1] = NA

In [665]:
df.iloc[4:, 2] = NA

In [666]:
df

,0,1,2
0,-0.831154,-2.370232,-1.860761
1,-0.860757,0.560145,-1.265934
2,0.119827,NaN,0.332883
3,-2.359419,NaN,-1.541996
4,-0.970736,NaN,NaN
5,0.377984,NaN,NaN


In [667]:
df.fillna(method="ffill")

,0,1,2
0,-0.831154,-2.370232,-1.860761
1,-0.860757,0.560145,-1.265934
2,0.119827,0.560145,0.332883
3,-2.359419,0.560145,-1.541996
4,-0.970736,0.560145,-1.541996
5,0.377984,0.560145,-1.541996


In [668]:
df.fillna(method="ffill", limit=2)

,0,1,2
0,-0.831154,-2.370232,-1.860761
1,-0.860757,0.560145,-1.265934
2,0.119827,0.560145,0.332883
3,-2.359419,0.560145,-1.541996
4,-0.970736,NaN,-1.541996
5,0.377984,NaN,-1.541996


In [669]:
# 조금만 창의적으로 생각하면 fillna를 이용해서 매우 다양한 일을 할 수 있는데 예를 들어 Series의 평균값이나 중간값을 전달할 수도 있다.

In [670]:
data = pd.Series([1., NA, 3.5, NA, 7])

In [671]:
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

In [672]:
# fillna에 대한 설명은 [표 7-2]를 참조하자. [표 7-2] fillna 함수 인자 - 페이지 276

In [673]:
# 7.2 데이터 변형

In [674]:
# 지금까지 데이터를 재배치하는 방법을 알아봤다. 필터링, 정제 및 다른 변형 역시 중요한 연산이다.

In [675]:
# 7.2.1 중복 제거하기

In [676]:
# 여러 가지 이유로 DataFrame에서 중복된 로우를 발견할 수 있다. 예제를 보자.

In [677]:
data = pd.DataFrame({"k1": ["one", "two"] * 3 + ["two"],
                     "k2" : [1, 1, 2, 3, 3, 4, 4]})

In [678]:
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [679]:
# DataFrame의 duplicated 메서드는 각 로우가 중복인지 아닌지 알려주는 불리언 Series를 반환한다.

In [680]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [681]:
# drop_duplicates는 duplicated 배열이 False인 DataFrame을 반환한다.

In [682]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [683]:
# 이 두 메서드는 기본적으로 모든 컬럼에 적용되며 중복을 찾아내기 위한 부분합을 따로 지정해줄 수도 있다. 

In [684]:
# 새로운 컬럼을 하나 추가하고 "k1" 컬럼에 기반해서 중복을 걸러내려면 다음과 같이 한다.

In [685]:
data["v1"] = range(7)

In [686]:
data.drop_duplicates(["k1"])

,k1,k2,v1
0,one,1,0
1,two,1,1


In [687]:
# duplicated와 drop_duplicates는 기본적으로 처음 발견된 값을 유지한다. 

In [688]:
# keep="last" 옵션을 넘기면 마지막으로 발견된 값을 반환한다.

In [689]:
data.drop_duplicates(["k1", "k2"], keep="last")

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


In [690]:
# 7.2.2 함수나 매핑을 이용해서 데이터 변형하기

In [691]:
# 데이터를 다루다 보면 DataFrame의 컬럼이나 Series, 배열 내의 값을 기반으로 데이터의 형태를 변환하고 싶은 경우가 있다.

In [692]:
# 가상으로 수집한 육류에 대한 다음 정보를 한번 살펴보자.

In [693]:
data = pd.DataFrame({"food" : ["bacon", "pulled pork", "bacon",
                               "Pastrami", "corned beef", "Bacon",
                               "pastrami", "honey ham", "nova lox"],
                     "ounces" : [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})

In [694]:
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [695]:
# 그리고 해당 육류가 어떤 동물의 고기인지 알려줄 수 있는 컬럼을 하나 추가한다고 가정하고 육류별 동물을 담고 있는 사전 데이터를 아래처럼 작성하자.

In [696]:
meat_to_animal = {
    "bacon": "pig",
    "pulled pork": "pig",
    "pastrami": "cow", 
    "corned beef": "cow",
    "honey ham": "pig",
    "nova lox": "salmon" 
}

In [697]:
# Series의 map 메서드는 사전류의 객체나 어떤 함수를 받을 수 있는데, 위 데이터에는 육류 이름에 대소문자가 섞여 있는 사소한 문제가 있으므로 
# str.lower 메서드를 사용해서 모두 소문자로 변경한다.

In [698]:
lowercased = data["food"].str.lower()

In [699]:
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [700]:
data["animal"] = lowercased.map(meat_to_animal)

In [701]:
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [702]:
# 물론 함수를 넘겨서 같은 일을 수행할 수도 있다.

In [703]:
data["food"].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

In [704]:
# map 메서드를 사용하면 데이터의 요소별 변환 및 데이터를 다듬는 작업을 편리하게 수행할 수 있다.

In [705]:
# 7.2.3 값 치환하기

In [706]:
# fillna 메서드를 사용해서 누락된 값을 채우는 일은 일반적인 값 치환 작업이라고 볼 수 있다.

In [707]:
# 위에서 살펴봤듯이 map 메서드를 한 객체 안에서 값의 부분집합을 변경하는 데 사용했다면 replace 메서드는 같은 작업에 대해 좀 더 간단하고 유연한 방법을 제공한다. 

In [708]:
# 다음 Series 객체를 살펴보자.

In [709]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])

In [710]:
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [711]:
# -999는 누락된 데이터를 나타내기 위한 값이다. 

In [712]:
# replace 메서드를 이용하면 이 값을 pandas에서 인식할 수 있는 NA 값으로 치환한 새로운 Series를 생성할 수 있다. 인자로 inplace=True를 넘기지 않았다면

In [713]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [714]:
# 여러 개의 값을 한 번에 치환하려면 하나의 값 대신 치환하려는 값의 리스트를 넘기면 된다. 

In [715]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [716]:
# 치환하려는 값마다 다른 값으로 치환하려면 누락된 값 대신 리스트를 사용하면 된다. 

In [717]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [718]:
# 두 개의 리스트 대신 사전을 이용하는 것도 가능하다. 

In [719]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [720]:
# Note: data.replace 메서드는 문자열 치환을 항목 단위로 수행하는 data.str.replace와 구별되는데 자세한 내용은 나중에 Series의 문자열 메서드와 함께 알아보겠다.

In [721]:
# 7.2.4 축 색인 이름 바꾸기

In [722]:
# Series의 값들처럼 축 이름 역시 유사한 방식으로 함수나 새롭게 바꿀 값을 이용해서 변환할 수 있다. 

In [723]:
# 새로운 자료구조를 만들지 않고 그 자리에서 바로 축 이름을 변경하는 것이 가능하다. 

In [724]:
# 다음 예제를 살펴보자. 

In [725]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=["Ohio", "Colorado", "New York"],
                    columns=["one", "two", "three", "four"])

In [726]:
# Series와 마찬가지로 축 색인에도 map 메서드가 있다. 

In [727]:
transform = lambda x: x[:4].upper()

In [728]:
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [729]:
# 대문자로 변경된 축 이름을 DataFrame의 index에 바로 대입할 수 있다. 

In [730]:
data.index = data.index.map(transform)

In [731]:
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [732]:
# 원래 객체를 변경하지 않고 새로운 객체를 생성하려면 rename 메서드를 사용한다. 

In [733]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [734]:
# 특히 rename 메서드는 사전 형식의 객체를 이용해서 축 이름 중 일부만 변경하는 것도 가능하다. 

In [735]:
data.rename(index={"OHIO": "INDIANA"},
            columns={"three": "peekaboo"})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [736]:
# rename 메서드를 사용하면 DataFrame을 직접 복사해서 index와 columns 속성을 갱신할 필요 없이 바로 변경할 수 있다. 

In [737]:
# 원본 데이터를 바로 변경하려면 inplace=True 옵션을 넘겨주면 된다.

In [738]:
data.rename(index={"OHIO": "INDIANA"}, inplace=True)

In [739]:
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [740]:
# 7.2.5 개별화와 양자화

In [741]:
# 연속성 데이터는 종종 개별로 분할하거나 아니면 분석을 위해 그룹별로 나누기도 하는데, 수업에 참여하는 학생 그룹 데이터가 있고, 나이대에 따라 분류한다고 가정하자. 

In [742]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [743]:
# 이 데이터를 pandas의 cut 함수를 이용해서 18-25, 26-35, 35-60, 60 이상 그룹으로 나누어보자. 

In [744]:
bins = [18, 25, 35, 60, 100]

In [745]:
cats = pd.cut(ages, bins)

In [746]:
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [747]:
# pandas에서 반환하는 객체는 Categorical이라는 특수한 객체다. 결과에서 보이는 그룹은 pandas.cut으로 계산된 것이다. 

In [748]:
# 이 객체는 그룹 이름이 담긴 배열이라고 생각하면 된다. 

In [749]:
# 이 Categorical 객체는 codes 속성에 있는 ages 데이터에 대한 카테고리 이름을 categories라는 배열에 내부적으로 담고 있다. 

In [750]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [751]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]], dtype='interval[int64, right]')

In [752]:
pd.value_counts(cats)

(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
dtype: int64

In [753]:
# pd.value_counts(cats)는 pandas.cut 결과에 대한 그룹 수다. 

In [754]:
# 간격을 나타내는 표기법은 중괄호로 시작해서 대괄호로 끝나는데 "중괄호 쪽의 값은 포함되지 않고 대괄호 쪽의 값은 포함"하는 간격을 나타낸다. 

In [755]:
# right=False를 넘겨서 중괄호 대신 대괄호 쪽이 포함되지 않도록 바꿀 수 있다. 

In [756]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64, left]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

In [757]:
# labels 옵션으로 그룹의 이름을 직접 넘겨줄 수도 있다. 

In [758]:
groups_names = ["Youth", "YoungAdult", "MiddleAged", "Senior"]

In [759]:
pd.cut(ages, bins, labels=groups_names)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

In [760]:
# 만약 cut 함수에 명시적으로 그룹의 경곗값을 넘기지 않고 그룹의 개수를 넘겨주면 데이터에서 최솟값과 최댓값을 기준으로 균등한 길이의 그룹을 자동으로 계산한다.

In [761]:
# 어떤 균등분포 내에서 4개의 그룹으로 나누는 경우를 생각해보자.

In [762]:
data = np.random.rand(20)

In [763]:
pd.cut(data, 4, precision=2)

[(0.79, 0.97], (0.6, 0.79], (0.79, 0.97], (0.41, 0.6], (0.22, 0.41], ..., (0.6, 0.79], (0.22, 0.41], (0.41, 0.6], (0.22, 0.41], (0.22, 0.41]]
Length: 20
Categories (4, interval[float64, right]): [(0.22, 0.41] < (0.41, 0.6] < (0.6, 0.79] < (0.79, 0.97]]

In [764]:
# precision=2 옵션은 소수점 아래 2자리까지로 제한한다. 

In [765]:
# 이를 위한 가장 적합한 함수로 qcut이 있는데 표본 변위치를 기반으로 데이터를 나눠준다. 

In [766]:
# cut 함수를 사용하면 데이터의 분산에 따라 각각의 그룹마다 데이터 수가 다르게 나뉘는 경우가 많다. 

In [767]:
# qcut은 표준 변위치를 사용하기 때문에 적당히 같은 크기의 그룹으로 나눌 수 있다.

In [768]:
data = np.random.randn(1000) # 정규 분포

In [769]:
cats = pd.qcut(data, 4)

In [770]:
cats

[(0.62, 3.928], (-2.9499999999999997, -0.684], (0.62, 3.928], (0.62, 3.928], (0.62, 3.928], ..., (-2.9499999999999997, -0.684], (0.62, 3.928], (-0.0265, 0.62], (-0.0265, 0.62], (-2.9499999999999997, -0.684]]
Length: 1000
Categories (4, interval[float64, right]): [(-2.9499999999999997, -0.684] < (-0.684, -0.0265] < (-0.0265, 0.62] < (0.62, 3.928]]

In [771]:
pd.value_counts(cats)

(-2.9499999999999997, -0.684]    250
(-0.684, -0.0265]                250
(-0.0265, 0.62]                  250
(0.62, 3.928]                    250
dtype: int64

In [772]:
# cut 함수와 유사하게 변위치를 직접 지정해줄 수 있다(변위치는 0부터 1까지다).

In [773]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

[(1.287, 3.928], (-2.9499999999999997, -1.187], (-0.0265, 1.287], (-0.0265, 1.287], (-0.0265, 1.287], ..., (-2.9499999999999997, -1.187], (-0.0265, 1.287], (-0.0265, 1.287], (-0.0265, 1.287], (-1.187, -0.0265]]
Length: 1000
Categories (4, interval[float64, right]): [(-2.9499999999999997, -1.187] < (-1.187, -0.0265] < (-0.0265, 1.287] < (1.287, 3.928]]

In [774]:
# 그룹 분석과 변위치를 다룰 때는 cut과 qcut 함수 같은 이산함수가 특히 더 유용한데 이 내용은 수집과 그룹 연산을 다루는 10장에서 다시 한번 살펴보도록 하겠다.

In [775]:
# 7.2.6 특잇값을 찾고 제외하기

In [776]:
# 배열 연산을 수행할 때는 특잇값을 제외하거나 적당한 값으로 대체하는 것이 중요하다. 

In [777]:
# 적절히 분산된 값이 담겨 있는 DataFrame을 살펴보자.

In [778]:
data = pd.DataFrame(np.random.randn(1000, 4))

In [779]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.045383,0.024079,-0.004794,-0.050389
std,0.997796,1.008737,0.996599,0.996706
min,-3.645860,-3.184377,-3.745356,-3.428254
25%,-0.600254,-0.612162,-0.697084,-0.747478
50%,0.040150,-0.013609,-0.029924,-0.088274
75%,0.746527,0.690847,0.699046,0.623331
max,2.653656,3.525865,2.735527,3.366626


In [780]:
# 이 DataFrame의 한 컬럼에서 절댓값이 3을 초과하는 값을 찾아내자. 

In [781]:
col = data[2]

In [782]:
col[np.abs(col) > 3]

38    -3.399312
133   -3.745356
Name: 2, dtype: float64

In [783]:
# 절댓값이 3을 초과하는 값이 들어 있는 모든 로우를 선택하려면 불리언 DataFrame에서 any 메서드를 사용하면 된다.

In [784]:
data[(np.abs(data) > 3).any(1)]

C:\Users\ahnjh\AppData\Local\Temp\ipykernel_25852\2969116195.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  data[(np.abs(data) > 3).any(1)]


,0,1,2,3
38,0.457246,-0.025907,-3.399312,-0.974657
57,1.951312,3.260383,0.963301,1.201206
133,0.508391,-0.196713,-3.745356,-1.520113
232,-0.242459,-3.056990,1.918403,-0.578828
255,0.682841,0.326045,0.425384,-3.428254
319,1.179227,-3.184377,1.369891,-1.074833
541,-3.548824,1.553205,-2.186301,1.277104
632,-0.578093,0.193299,1.397822,3.366626
779,-0.207434,3.525865,0.283070,0.544635
800,-3.645860,0.255475,-0.549574,-1.907459


In [785]:
# 이 기준대로 쉽게 값을 선택할 수 있으며, 아래 코드로 -3이나 3을 초과하는 값을 -3 또는 3으로 지정할 수 있다.

In [786]:
data[np.abs(data) > 3] = np.sign(data) * 3

In [787]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.046577,0.023534,-0.003650,-0.050328
std,0.993777,1.005496,0.992788,0.994152
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.600254,-0.612162,-0.697084,-0.747478
50%,0.040150,-0.013609,-0.029924,-0.088274
75%,0.746527,0.690847,0.699046,0.623331
max,2.653656,3.000000,2.735527,3.000000


In [788]:
# np.sign(data)는 data 값이 양수인지 음수인지에 따라 1이나 -1이 담긴 배열을 반환한다. 

In [789]:
np.sign(data).head()

,0,1,2,3
0,-1.0,-1.0,1.0,-1.0
1,-1.0,1.0,-1.0,-1.0
2,1.0,-1.0,1.0,-1.0
3,1.0,-1.0,1.0,-1.0
4,-1.0,-1.0,-1.0,-1.0


In [790]:
# 7.2.7 치환과 임의 샘플링

In [791]:
# numpy.random.permutaion 함수를 이용하면 Series나 DataFrame의 로우를 쉽게 임의 순서로 재배치할 수 있다. 

In [792]:
# 순서를 바꾸고 싶은 만큼의 길이를 permutaion 함수로 넘기면 바뀐 순서가 담긴 정수 배열이 생성된다. 

In [793]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))

In [794]:
sampler = np.random.permutation(5)

In [795]:
sampler

array([2, 4, 1, 0, 3])

In [796]:
# 이 배열은 iloc 기반의 색인이나 take 함수에서 사용 가능하다. 

In [797]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [798]:
df.take(sampler)

,0,1,2,3
2,8,9,10,11
4,16,17,18,19
1,4,5,6,7
0,0,1,2,3
3,12,13,14,15


In [799]:
# 치환 없이 일부만 임의로 선택하려면 Series나 DataFrame의 sample 메서드를 사용하면 된다. 

In [800]:
df.sample(n=3)

,0,1,2,3
2,8,9,10,11
4,16,17,18,19
1,4,5,6,7


In [801]:
# (반복 선택을 허용하며) 표본을 치환을 통해 생성해내려면 sample에 replace=True 옵션을 넘긴다. 

In [802]:
choices = pd.Series([5, 7, -1, 6, 4])

In [803]:
draws = choices.sample(n=10, replace=True)

In [804]:
draws

4    4
1    7
2   -1
3    6
4    4
1    7
4    4
0    5
3    6
0    5
dtype: int64

In [805]:
# 7.2.8 표시자/더미 변수 계산하기

In [806]:
# 통계 모델이나 머신 러닝 애플리케이션을 위한 또 다른 데이터 변환은 분류값을 "더미"나 "표시자" 행렬로 전환하는 것이다. 

In [807]:
# 만약 어떤 DataFrame의 한 컬럼에 k가지 값이 있다면 k개의 컬럼이 있는 DataFrame이나 행렬을 만들고 값으로는 1과 0을 채워 넣을 것이다.

In [808]:
# pandas의 get_dummies가 이를 위한 함수인데 학습자 스스로 새로운 방법을 고안해내는 것도 어렵지 않을 것이다. 

In [809]:
# 앞서 살펴본 DataFrame을 다시 살펴보자.

In [810]:
df = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"],
                   "data1": range(6)})

In [811]:
pd.get_dummies(df["key"])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [812]:
# 표시자 DataFrame 안에 있는 컬럼에 접두어를 추가한 후 다른 데이터와 병합하고 싶을 경우가 있다. 

In [813]:
# get_dummies 함수의 prefix(접두어) 인자를 사용하면 이를 수행할 수 있다. 

In [814]:
dummies = pd.get_dummies(df["key"], prefix="key")

In [815]:
df_with_dummy = df[["data1"]].join(dummies)

In [816]:
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [817]:
# DataFrame의 한 로우가 여러 카테고리에 속한다면 조금 복잡해지는데, 자세한 내용은 14.2절 "MovieLens의 영화 평점 데이터"에서 살펴보자.

In [818]:
# 우선은 각주 처리하고, 14장이 되면 그때 다시 코드를 적어보겠다.

In [819]:
# names = ["movie_id", "title", "genres"]

In [820]:
# movies = pd.read_table("datasets/movielens/movies.dat", sep="::",
#                         header=None, names=mnames)

In [821]:
# movies[:10]

In [822]:
# 각 장르마다 표시자 값을 추가하려면 약간의 수고를 해야 하는데 먼저 데이터 묶음에서 유일한 장르 목록을 추출해야 한다. 

In [823]:
# all_genres = []

In [824]:
# for x in movies.genres:
#   all_genres.extend(x.split("|"))

In [825]:
# genres = pd.unique(all_genres)

In [826]:
# 이제 장르는 아래와 같은 모양이 된다. 

In [827]:
# genres

In [828]:
# 이제 표시자 DataFrame을 생성하기 위해 0으로 초기화된 DataFrame을 생성하자. 

In [829]:
# zero_matrix = np.zeros((len(movies), len(genres)))

In [830]:
# dummies = pd.DataFrame(zero_matrix, columns=genres)

In [831]:
# 각 영화를 순회하면서 dummies의 각 로우의 항목을 1로 설정한다. 

In [832]:
# 각 장르의 컬럼 색인을 계산하기 위해 dummies.columns를 사용하자. 

In [833]:
# gen = movies.genres[0]

In [834]:
# gen.split("|")

In [835]:
# dummies.columns.get_indexer(gen.split("|"))

In [836]:
# 그리고 .iloc를 이용해서 색인에 맞게 값을 대입하자. 

In [837]:
# for i, gen in enumerate(movies.genres):
#   indices = dummies.columns.get_indexer(gen.split("|"))
#   dummies.iloc[i, indices] = 1

In [838]:
# 그리고 앞에서 한 것처럼 movies와 조합하면 된다. 

In [839]:
# movies_windic = movies.join(dummies.add_prefix("Genre_"))

In [840]:
# movies_windic.iloc[0]

In [841]:
# Note : 이보다 더 큰 데이터라면 이 방법으로 다중 멤버십을 갖는 표시자 변수를 생성하는 것은 그다지 빠른 방법은 아니다. 
# 속도를 높이려면 직접 NumPy 배열에 접근하는 저수준의 함수를 작성해서 DataFrame에 결과를 저장하도록 해야 한다. 

In [842]:
# get_dummies와 cut 같은 이산함수를 잘 조합하면 통계 애플리케이션에서 유용하게 사용할 수 있다. 

In [843]:
np.random.seed(12345)

In [844]:
values = np.random.rand(10)

In [845]:
values

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [846]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]

In [847]:
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


In [848]:
# 여기서는 예제 값이 불변하도록 numpy.random.seed 함수를 이용해서 난수 시드값을 지정했다. 

In [849]:
# pandas.get_dummies 함수는 나중에 다시 살펴볼 것이다. 

In [850]:
# 7.3 문자열 다루기

In [851]:
# 파이썬은 문자열이나 텍스트 처리의 용이함 덕분에 원시 데이터를 처리하는 인기 있는 언어가 되었다. 

In [852]:
# 대부분의 텍스트 연산은 문자열 객체의 내장 메서드로 간단하게 처리할 수 있다.

In [853]:
# 좀 더 복잡한 패턴 매칭이나 텍스트 조작은 정규 표현식을 필요로 한다. 

In [854]:
# pandas는 배열 데이터 전체에 쉽게 정규 표현식을 적용하고, 누락된 데이터를 편리하게 처리할 수 있는 기능을 포함하고 있다. 

In [855]:
# 7.3.1 문자열 객체 메서드

In [856]:
# 문자열을 다뤄야 하는 대부분의 애플리케이션은 내장 문자열 메서드만으로도 충분하다. 

In [857]:
# 예를 들어 쉼표로 구분된 문자열은 split 메서드를 이용해서 분리할 수 있다. 

In [858]:
val = "a, b, guido"

In [859]:
val.split(",")

['a', ' b', ' guido']

In [860]:
# split 메서드는 종종 공백 문자(줄바꿈 문자 포함)를 제거하는 strip 메서드와 조합해서 사용하기도 한다. 

In [861]:
pieces = [x.strip() for x in val.split(",")]

In [862]:
pieces

['a', 'b', 'guido']

In [863]:
# 이렇게 분리된 문자열은 더하기 연산을 사용해서 :: 문자열과 합칠 수도 있다. 

In [864]:
first, second, third = pieces

In [865]:
first + "::" + second + "::" + third

'a::b::guido'

In [866]:
# 하지만 이 방법은 실용적이면서 범용적인 메서드는 아니다. 빠르고 좀 더 파이썬스러운 방법은 리스트나 튜플을 :: 문자열의 join 메서드로 전달하는 것이다.

In [867]:
"::".join(pieces) # 좀 더 효율적인 코드

'a::b::guido'

In [868]:
# 일치하는 부분문자열의 위치를 찾는 방법도 있다. index나 find를 사용하는 것도 가능하지만 파이썬의 in 예약어를 사용하면 일치하는 부분문자열을 쉽게 찾을 수 있다. 

In [869]:
"guido" in val

True

In [870]:
val.index(",")

1

In [871]:
val.find(":")

-1

In [872]:
# find와 index의 차이점은 index의 경우 문자열을 찾지 못하면 예외를 발생시킨다는 것이다. 

In [873]:
# find의 경우에는 -1을 반환한다. 

In [874]:
val.index(":") # ValueError 발생

ValueError: substring not found

In [875]:
# count는 특정 부분문자열이 몇 건 발견되었는지 반환한다. 

In [876]:
val.count(",")

2

In [877]:
# replace는 찾아낸 패턴을 다른 문자열로 치환한다.

In [878]:
# 이 메서드는 대체할 문자열로 비어 있는 문자열을 넘겨서 패턴을 삭제하기 위한 방법으로 자주 사용되기도 한다. 

In [879]:
val.replace(",", "::")

'a:: b:: guido'

In [880]:
val.replace(",", "")

'a b guido'

In [881]:
# [표7-3]에 파이썬의 문자열 함수를 모아두었다. 페이지 298쪽

In [882]:
# 정규 표현식을 이런 다양한 용도로 사용할 수 있다. 

In [883]:
# 7.3.2 정규 표현식

In [884]:
# 정규 표현식은 텍스트에서 문자열 패턴을 찾는 유연한 방법을 제공한다. 흔히 regex라 불리는 단일 표현식은 정규 표현 언어로 구성된 문자열이다. 

In [885]:
# 파이썬에는 re 모듈이 내장되어 있어서 문자열에 대한 정규 표현식을 처리한다. 몇 가지 예제로 알아보자

In [886]:
# NOTE_ 정규 표현식을 작성하는 방법은 그 자체로 하나의 독립된 장으로 구성할 수 있고 따라서 이 책에서 다루는 범위를 벗어난다. 
# 정규 표현식에 관한 많은 튜토리얼과 레퍼런스가 존재한다. 

In [887]:
# re 모듈 함수는 패턴 매칭, 치환, 분리, 세 가지로 나눌 수 있다. 물론 이 세 가지는 모두 서로 연관되어 있다.

In [888]:
# 정규 표현식은 텍스트 내에 존재하는 패턴을 표현하고 이를 여러 가지 다양한 목적으로 사용할 수 있도록 되어 있다. 

In [889]:
# 간단한 예제를 살펴보자. 여러가지 공백 문자(탭, 스페이스, 개행 문자)가 포함된 문자열을 나누고 싶다면 하나 이상의 공백 문자를 의미하는 \s+를 사용해서 문자열을 분리한다. 

In [890]:
import re

In [891]:
text = "foo bar\t baz \tqux"

In [892]:
re.split("\s+", text)

['foo', 'bar', 'baz', 'qux']

In [893]:
# re.split("\s+", text)를 사용하면 먼저 정규 표현식이 컴파일 되고 그 다음에 split 메서드가 실행된다. 

In [894]:
# re.compile로 직접 정규 표현식을 컴파일하고 그렇게 얻은 정규 표현식 객체를 재사용하는 것도 가능하다. 

In [895]:
regex = re.compile("\s+")

In [896]:
regex.split(text)

['foo', 'bar', 'baz', 'qux']

In [897]:
# 정규 표현식에 매칭되는 모든 패턴의 목록을 얻고 싶다면 findall 메서드를 사용한다. 

In [898]:
regex.findall(text)

[' ', '\t ', ' \t']

In [899]:
# NOTE_ 정규 표현식에서 \ 문자가 이스케이프되는 문제를 피하려면 raw 문자열 표기법을 사용한다. 
# 그러면 \를 이스케이프 문자로 처리하지 않고 일반 문자로 처리하기 때문에 \를 간단하게 표현할 수 있다. 
# 즉, "C:\\x" 대신 r"C:\x"를 사용한다. 

In [900]:
# 같은 정규 표현식을 다른 문자열에도 적용해야 한다면 re.compile을 이용해서 정규 표현식 객체를 만들어 쓰는 방법을 추천한다. 

In [901]:
# 이렇게 하면 CPU 사용량을 아낄 수 있다.

In [902]:
# match와 search는 findall 메서드와 관련이 있다. findall은 문자열에서 일치하는 모든 부분문자열을 찾아주지만 search 메서드는 패턴과 일치하는 첫 번째 존재를 반환한다. 

In [903]:
# match 메서드는 이보다 더 엄격해서 문자열의 시작부분에서 일치하는 것만 찾아준다. 약간 복잡한 예제로 이메일 주소를 검사하는 정규 표현식을 한 번 살펴보자.

In [920]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""

pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

# re.IGNORECASE는 정규 표현식이 대소문자를 가리지 않도록 한다. 
regex = re.compile(pattern, flags=re.IGNORECASE)

In [921]:
# findall 메서드를 사용해서 이메일 주소의 리스트를 생성하자. 

In [922]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

In [923]:
# search는 텍스트에서 첫 번째 이메일 주소만을 찾아준다.

In [924]:
# 위 정규 표현식에 대한 match 객체는 그 정규 표현 패턴이 문자열 내에서 위치하는 시작점과 끝점만을 알려준다.

In [925]:
m = regex.search(text)

In [926]:
m

<re.Match object; span=(5, 20), match='dave@google.com'>

In [927]:
text[m.start():m.end()]

'dave@google.com'

In [928]:
# regex.match는 None을 반환한다. 왜냐하면 그 정규 표현 패턴이 문자열의 시작점에서부터 일치하는지 검사하기 때문이다. 

In [929]:
print(regex.match(text))

None


In [930]:
# sub 메서드는 찾은 패턴을 주어진 문자열로 치환하여 새로운 문자열을 반환한다. 

In [931]:
print(regex.sub("REDACTED", text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



In [932]:
# 이메일 주소를 찾아서 동시에 각 이메일 주소를 사용자 이름, 도메인 이름, 도메인 접미사 세가지 컴포넌트로 나눠야한다면 각 패턴을 괄호로 묶어준다.

In [939]:
pattern = r"([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})"

In [940]:
regex = re.compile(pattern, flags=re.IGNORECASE)

In [941]:
# 이렇게 만든 match 객체를 이용하면 group 메서드로 각 패턴 컴포넌트의 튜플을 얻을 수 있다. 

In [942]:
m = regex.match("wesm@bright.net")

In [944]:
m.groups() # AttributeError가 발생하여 저자의 깃허브를 참조했다.

('wesm', 'bright', 'net')

In [945]:
# 패턴에 그룹이 존재한다면 findall 메서드는 튜플의 목록을 반환한다.

In [946]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

In [947]:
# sub 역시 마찬가지로 \1, \2 같은 특수한 기호를 사용해서 각 패턴 그룹에 접근할 수 있다. \1은 첫 번째로 찾은 그룹을 의미하고, \2는 두 번째로 찾은 그룹을 의미한다.

In [949]:
print(regex.sub(r"Username: \1, Domain: \2, Suffix: \3", text))

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



In [951]:
# 이 밖에도 파이썬에서 할 수 있는 정규 표현식은 많이 있지만 대부분 이 책의 범위를 벗어난다. 표 [7-4]에 몇 가지 메서드를 소개했다. 페이지 302

In [952]:
# 7.3.3 pandas의 벡터화된 문자열 함수

In [953]:
# 뒤죽박죽인 데이터를 분석을 위해 정리하는 일은 문자열을 다듬고 정규화하는 작업을 필요로 한다. 

In [954]:
# 문자열을 담고 있는 컬럼에 누락된 값이 있다면 일을 더 복잡하게 만든다.

In [955]:
data = {"Dave": 'dave@google.com', "Steve": "steve@gmail.com",
        "Rob": "rob@gmail.com", "Wes": np.nan}

In [956]:
data = pd.Series(data)

In [957]:
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [958]:
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

In [959]:
# 문자열과 정규 표현식 메서드는 data.map을 사용해서 각 값에 적용(lambda 혹은 다른 함수를 넘겨서)할 수 있지만 NA 값을 만나면 실패하게 된다. 

In [960]:
# 이런 문제에 대처하기 위해 Series에는 NA 값을 건너뛰도록 하는 간결한 문자열 처리 메서드가 있다.

In [961]:
# 이는 Series의 Str 속성을 이용하는데, 예를 들어 각 이메일 주소가 "gmail"을 포함하고 있는지 str.contains를 이용해서 검사할 수 있다.

In [962]:
data.str.contains("gmail")

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

In [963]:
# 정규 표현식을 IGNORECASE 같은 re 옵션과 함께 사용하는 것도 가능하다. 

In [964]:
pattern

'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2,4})'

In [965]:
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

In [966]:
# 벡터화된 요소를 꺼내오는 몇 가지 방법이 있는데 str.get을 이용하거나 str 속성의 색인을 이용한다.

In [972]:
matches = data.str.match(pattern, flags=re.IGNORECASE)

In [973]:
matches

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

In [969]:
# 내재된 리스트의 원소에 접근하기 위해서는 색인을 넘기면 된다. 

In [1]:
matches.str.get(1) # AttributeError: Can only use .str accessor with string values! 오류 발생

NameError: name 'matches' is not defined

In [ ]:
# 아래 문법으로 문자열을 잘라낼 수도 있다.

In [ ]:
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object

In [ ]:
# [표7-5]에 pandas의 문자열 메서드를 정리해두었다. 페이지 305-306